In [ ]:
from numpy.random import seed
seed(1)
#from tensorflow.random import set_seed
#set_seed(2)

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from nltk.tokenize import sent_tokenize,word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import nltk
from nltk.corpus import stopwords

from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import StandardScaler

import pickle

nltk.download('punkt')
nltk.download('stopwords')
#np.random.seed(1)
random_state = 7

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#train = pd.read_csv('/content/drive/My Drive/Author_identification/train.csv')
#test = pd.read_csv('/content/drive/My Drive/Author_identification/test.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv('./Data/Author_identification/train.csv')
test = pd.read_csv('./Data/Author_identification/test.csv')

In [ ]:
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [ ]:
test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [ ]:
train['sentences'] = train.text.transform(lambda x: len(sent_tokenize(x)))
train['words'] = train.text.transform(lambda x: len(word_tokenize(x)))
train['text_length'] = train.text.transform(lambda x: len(x))

text_info = train.groupby("author")[['sentences','words','text_length']].sum()
text_info

,sentences,words,text_length
author,,,
EAP,8206,232184,1123585
HPL,5876,173979,878178
MWS,6128,188824,916632


In [ ]:
stop_words = set(stopwords.words('english')).union(set([',','.','?','!',':',';',"'",'"','-',"''","`","``"]))
author_dict = {'EAP': 0, 'HPL': 1, 'MWS': 2}

In [ ]:
new_text = []
new_author = []
for i, row in train[['text','author']].iterrows():
    word_tokens = word_tokenize(row['text'])
    new_text.append(" ".join([w for w in word_tokens if not w in stop_words]))
    new_author.append(author_dict[row['author']])
new_train = pd.DataFrame(data={'text': new_text, 'author': new_author})

In [ ]:
new_test_text = []
for i, row in test[['text']].iterrows():
    word_tokens = word_tokenize(row['text'])
    new_test_text.append(" ".join([w for w in word_tokens if not w in stop_words]))
new_test = pd.DataFrame(data={'text': new_text})

In [ ]:
X = new_train['text']
y = new_train['author']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=random_state)
X_test = new_test['text']

In [ ]:
new_text_ws = [] # without stopwords
for i, row in train[['text']].iterrows():
    word_tokens = word_tokenize(row['text'])
    new_text_ws.append(" ".join([w for w in word_tokens if not w in [',','.','?','!',':',';',"'",'"','-',"''","`","``"]]).lower())
new_train_ws = pd.DataFrame(data={'text': new_text_ws, 'author': new_author})

In [ ]:
X_ws = new_train_ws['text']
y_ws = new_train_ws['author']
X_ws_train, X_ws_valid, y_ws_train, y_ws_valid = train_test_split(X_ws, y_ws, test_size=0.3, random_state=random_state)

In [ ]:
X_ws.head()

0    this process however afforded me no means of a...
1    it never once occurred to me that the fumbling...
2    in his left hand was a gold snuff box from whi...
3    how lovely is spring as we looked from windsor...
4    finding nothing else not even gold the superin...
Name: text, dtype: object

In [ ]:
new_train

,text,author
0,This process however afforded means ascertaini...,0
1,It never occurred fumbling might mere mistake,1
2,In left hand gold snuff box capered hill cutti...,0
3,How lovely spring As looked Windsor Terrace si...,2
4,Finding nothing else even gold Superintendent ...,1
...,...,...
19574,I could fancied I looked eminent landscape pai...,0
19575,The lids clenched together spasm,0
19576,Mais il faut agir say Frenchman never faints o...,0
19577,For item news like strikes us coolly received,0


In [ ]:
X_train_list = list(X_train)
y_train_list = list(y_train)
X_valid_list = list(X_valid)
y_valid_list = list(y_valid)

In [ ]:
X_ws_train_list = list(X_ws_train)
y_ws_train_list = list(y_ws_train)
X_ws_valid_list = list(X_ws_valid)
y_ws_valid_list = list(y_ws_valid)

In [ ]:
y_ws_train_list = np.array(y_ws_train_list)
y_ws_valid_list = np.array(y_ws_valid_list)

In [ ]:
y_ws_train_list[:50]

array([0, 0, 1, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 1, 1, 0, 1, 0, 1, 2, 0, 1, 1, 2, 0,
       0, 1, 2, 2, 2, 0])

In [ ]:
X_ws_train_list[0].split()[0]

'should'

Word2vec vectors for each author

In [ ]:
import multiprocessing
from gensim.models import Word2Vec

data_0 = new_train_ws.loc[new_train_ws['author'] == 0]
data_1 = new_train_ws.loc[new_train_ws['author'] == 1]
data_2 = new_train_ws.loc[new_train_ws['author'] == 2]

data_0 = data_0.sample(frac=1).reset_index(drop=True)
data_1 = data_1.sample(frac=1).reset_index(drop=True)
data_2 = data_2.sample(frac=1).reset_index(drop=True)

train_0 = data_0[:7400]
train_1 = data_1[:5135]
train_2 = data_2[:5544]
test_0 = data_0[7400:]
test_1 = data_1[5135:]
test_2 = data_2[5544:]

train_0 = data_0[:5000]
train_1 = data_1[:5000]
train_2 = data_2[:5000]
test_0 = data_0[5000:]
test_1 = data_1[5000:]
test_2 = data_2[5000:]

In [ ]:
from gensim.models.phrases import Phrases, Phraser
sent_0 = [row.split() for row in train_0['text']]
phrases_0 = Phrases(sent_0, min_count=30, progress_per=10000)
bigram_0 = Phraser(phrases_0)
sentences_0 = bigram_0[sent_0]
w2v_model_0 = Word2Vec(min_count=2, window=2, size=300, sample=6e-5, alpha=0.03, min_alpha=0.0007, negative=20, workers=2)

from time import time
t = time()
w2v_model_0.build_vocab(sentences_0, progress_per=10000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

t = time()
w2v_model_0.train(sentences_0, total_examples=w2v_model_0.corpus_count, epochs=30, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

w2v_model_0.init_sims(replace=True)

w2v_model_0.wv.similarity('you', 'are')

Time to build vocab: 0.03 mins
Time to train the model: 0.34 mins


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.98576355

In [ ]:
from gensim.models.phrases import Phrases, Phraser
sent_1 = [row.split() for row in train_1['text']]
phrases_1 = Phrases(sent_1, min_count=30, progress_per=10000)
bigram_1 = Phraser(phrases_1)
sentences_1 = bigram_1[sent_1]
w2v_model_1 = Word2Vec(min_count=2, window=2, size=300, sample=6e-5, alpha=0.03, min_alpha=0.0007, negative=20, workers=2)

from time import time
t = time()
w2v_model_1.build_vocab(sentences_1, progress_per=10000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

t = time()
w2v_model_1.train(sentences_1, total_examples=w2v_model_1.corpus_count, epochs=30, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

w2v_model_1.init_sims(replace=True)

w2v_model_1.wv.similarity('you', 'are')

Time to build vocab: 0.03 mins
Time to train the model: 0.39 mins


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.9840455

In [ ]:
from gensim.models.phrases import Phrases, Phraser
sent_2 = [row.split() for row in train_2['text']]
phrases_2 = Phrases(sent_2, min_count=30, progress_per=10000)
bigram_2 = Phraser(phrases_2)
sentences_2 = bigram_2[sent_2]
w2v_model_2 = Word2Vec(min_count=2, window=2, size=300, sample=6e-5, alpha=0.03, min_alpha=0.0007, negative=20, workers=2)

from time import time
t = time()
w2v_model_2.build_vocab(sentences_2, progress_per=10000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

t = time()
w2v_model_2.train(sentences_2, total_examples=w2v_model_2.corpus_count, epochs=30, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

w2v_model_2.init_sims(replace=True)

w2v_model_2.wv.similarity('you', 'are')

Time to build vocab: 0.03 mins
Time to train the model: 0.37 mins


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.98244613

In [ ]:
word_vectors_0 = w2v_model_0.wv
word_vectors_1 = w2v_model_1.wv
word_vectors_2 = w2v_model_2.wv

Testing function: 
- input: test sentence
- output: the most likely author

In [ ]:
def testing(X,y):
  true = 0
  false = 0
  for index, a in X.items():
    #print(a)

    sim_0 = 0
    sim_1 = 0
    sim_2 = 0
    sims = []
    tokens = word_tokenize(a)
    lenght = len(tokens)
    for i in range(lenght-1):
      #print(i)
      if tokens[i] in word_vectors_0 and tokens[i+1] in word_vectors_0:
        sim = w2v_model_0.wv.similarity(tokens[i], tokens[i+1])
        sim_0 += sim*sim

      if tokens[i] in word_vectors_1 and tokens[i+1] in word_vectors_1:
        sim = w2v_model_1.wv.similarity(tokens[i], tokens[i+1])
        sim_1 += sim*sim

      if tokens[i] in word_vectors_2 and tokens[i+1] in word_vectors_2:
        sim = w2v_model_2.wv.similarity(tokens[i], tokens[i+1])
        sim_2 += sim*sim
    
    sims.append(sim_0/(lenght-1))
    sims.append(sim_1/(lenght-1))
    sims.append(sim_2/(lenght-1))
    
    #print(sims)
    #print(np.argmax(sims),y[index])
    if np.argmax(sims) == y[index]:
      true += 1
    else:
      false += 1
  print(true/(true+false))
  return (true, false)

In [ ]:
true_0, false_0 = testing(test_0['text'],test_0['author'])
true_1, false_1 = testing(test_1['text'],test_1['author'])
true_2, false_2 = testing(test_2['text'],test_2['author'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.7644827586206897
0.7307086614173228
0.6810344827586207


In [ ]:
true = true_0+ true_1 +true_2
false = false_0 + false_1 + false_2
print(true/(true+false))

0.7407730945621315


### Pre-trained fastText vectors

In [ ]:
# import gensim.downloader

In [ ]:
 #fasttext_vectors = gensim.downloader.load('fasttext-wiki-news-subwords-300')

[==================================================] 100.0% 958.5/958.4MB downloaded


In [ ]:
#with open('fasttext_vectors.pickle', 'wb') as f:
#    pickle.dump(fasttext_vectors, f)

In [ ]:
with open("fasttext_vectors.pickle", 'rb') as f:
    fasttext_vectors = pickle.load(f)

In [ ]:
np.array(fasttext_vectors.wv[X_ws_train_list[0].split()[0]])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


array([-1.0834e-02, -5.6953e-02, -3.1731e-02,  2.6749e-02,  4.7278e-02,
        4.2279e-03,  8.8756e-03, -8.3149e-02, -5.7365e-02, -4.3851e-02,
        1.0353e-02,  9.5677e-03,  1.5618e-02, -5.1344e-03,  2.6610e-02,
       -2.0032e-03,  1.2627e-01, -3.4053e-02,  7.2836e-02, -2.9336e-02,
        1.2728e-02, -5.9716e-02,  4.7661e-02,  8.5972e-02, -3.9190e-02,
       -7.3656e-03, -1.8250e-02,  2.8804e-02, -1.9875e-02, -2.4632e-02,
       -3.5466e-02, -2.4895e-02,  4.0472e-02,  6.5614e-02,  2.6882e-02,
       -8.2732e-02, -9.4424e-03, -2.4687e-02, -4.3293e-02,  1.7315e-02,
       -5.1753e-02, -1.6385e-02, -1.0530e-02, -1.6188e-02, -3.0672e-02,
       -1.8272e-03, -9.3336e-03, -1.5161e-02,  1.1667e-01, -4.2138e-04,
        9.3487e-03, -1.1487e-02, -5.4632e-02, -3.8202e-03, -2.2158e-01,
        3.0325e-02,  1.5373e-02,  1.8478e-02, -9.8686e-02, -2.1137e-02,
       -1.9092e-02,  1.4057e-02,  1.1103e-01, -2.9714e-02,  4.2807e-02,
       -5.1898e-02, -9.7448e-03,  5.6202e-03,  1.5102e-02, -7.24

#### Dense model with mean vectors

In each entry we collect the wordvectors of the tokens and take the mean of those (300 dimensional vectors).

In [ ]:
X_ws_train_ft_mean_vectors = np.empty([len(X_ws_train_list), 300])
for j in range(len(X_ws_train_list)):
    X_ws_train_ft_vectors = np.empty([len(X_ws_train_list[j].split()), 300])
    for i in range(len(X_ws_train_list[j].split())):
        try:
            X_ws_train_ft_vectors[i] = fasttext_vectors.wv[X_ws_train_list[j].split()[i]]
        except KeyError:
            X_ws_train_ft_vectors[i] = np.zeros(300)
    X_ws_train_ft_mean_vectors[j] = np.mean(X_ws_train_ft_vectors, axis=0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [ ]:
scaler = StandardScaler()
X_ws_train_ft_mean_vectors = scaler.fit_transform(X_ws_train_ft_mean_vectors)

In [ ]:
X_ws_valid_ft_mean_vectors = np.empty([len(X_ws_valid_list), 300])
for j in range(len(X_ws_valid_list)):
    X_ws_valid_ft_vectors = np.empty([len(X_ws_valid_list[j].split()), 300])
    for i in range(len(X_ws_valid_list[j].split())):
        try:
            X_ws_valid_ft_vectors[i] = fasttext_vectors.wv[X_ws_valid_list[j].split()[i]]
        except KeyError:
            X_ws_valid_ft_vectors[i] = np.zeros(300)
    X_ws_valid_ft_mean_vectors[j] = np.mean(X_ws_valid_ft_vectors, axis=0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [ ]:
scaler = StandardScaler()
X_ws_valid_ft_mean_vectors = scaler.fit_transform(X_ws_valid_ft_mean_vectors)

In [ ]:
len(X_ws_train_list)

13705

In [ ]:
from keras.utils import to_categorical
y_ws_train_list = to_categorical(y_ws_train_list)
y_ws_valid_list = to_categorical(y_ws_valid_list)

In [ ]:
y_ws_train_list[:5]

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [ ]:
dense_model = tf.keras.models.Sequential()
dense_model.add(tf.keras.Input(shape=(300,)))
dense_model.add(tf.keras.layers.Dense(300, activation='relu', kernel_initializer='he_normal'))
dense_model.add(Dropout(0.4))
dense_model.add(tf.keras.layers.Dense(300, activation='relu', kernel_initializer='he_normal'))
dense_model.add(tf.keras.layers.Dense(300, activation='relu', kernel_initializer='he_normal'))
dense_model.add(tf.keras.layers.Dense(3, activation='softmax'))
dense_model.output_shape

(None, 3)

In [ ]:
dense_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 300)               90300     
_________________________________________________________________
dropout (Dropout)            (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_2 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 903       
Total params: 271,803
Trainable params: 271,803
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.callbacks import EarlyStopping
early_stopping=EarlyStopping(monitor="val_accuracy",patience=10, verbose=1)

In [ ]:
dense_model.compile(optimizer=Adam(lr=1e-3),loss='categorical_crossentropy' ,metrics=['accuracy'])

In [ ]:
dense_model.fit(X_ws_train_ft_mean_vectors, y_ws_train_list,  validation_split = 0.2,
                     callbacks=[early_stopping], epochs=75)

Epoch 1/75
343/343 [==============================] - 2s 5ms/step - loss: 0.9153 - accuracy: 0.6111 - val_loss: 0.7406 - val_accuracy: 0.6830
Epoch 2/75
343/343 [==============================] - 2s 5ms/step - loss: 0.7329 - accuracy: 0.6927 - val_loss: 0.7106 - val_accuracy: 0.7059
Epoch 3/75
343/343 [==============================] - 2s 5ms/step - loss: 0.6633 - accuracy: 0.7242 - val_loss: 0.6955 - val_accuracy: 0.7096
Epoch 4/75
343/343 [==============================] - 2s 5ms/step - loss: 0.6210 - accuracy: 0.7401 - val_loss: 0.6591 - val_accuracy: 0.7231
Epoch 5/75
343/343 [==============================] - 2s 5ms/step - loss: 0.5925 - accuracy: 0.7581 - val_loss: 0.6695 - val_accuracy: 0.7143
Epoch 6/75
343/343 [==============================] - 2s 5ms/step - loss: 0.5552 - accuracy: 0.7715 - val_loss: 0.6572 - val_accuracy: 0.7231
Epoch 7/75
343/343 [==============================] - 2s 5ms/step - loss: 0.5347 - accuracy: 0.7840 - val_loss: 0.6526 - val_accuracy: 0.7289
Epoch 

In [ ]:
score = dense_model.evaluate(X_ws_valid_ft_mean_vectors, y_ws_valid_list, verbose = 0)
print("Test score: ", score[0])
print("Test accuracy: ", score[1])

Test score:  0.758577287197113
Test accuracy:  0.7410622835159302


### Dense model with the minmax vectors

Using the concatenation of the element-wise minimum and maximum of the wordvectors in each entry to get a vector (600 dimensional vectors)

In [ ]:
X_ws_train_ft_min_vectors = np.empty([len(X_ws_train_list), 300])
for j in range(len(X_ws_train_list)):
  X_ws_train_ft_vectors = np.empty([len(X_ws_train_list[j].split()), 300])
  for i in range(len(X_ws_train_list[j].split())):
    try:
      X_ws_train_ft_vectors[i] = fasttext_vectors.wv[X_ws_train_list[j].split()[i]]
    except KeyError:
      X_ws_train_ft_vectors[i] = np.zeros(300)
  X_ws_train_ft_min_vectors[j] = X_ws_train_ft_vectors.min(axis=0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [ ]:
X_ws_train_ft_max_vectors = np.empty([len(X_ws_train_list), 300])
for j in range(len(X_ws_train_list)):
  X_ws_train_ft_vectors = np.empty([len(X_ws_train_list[j].split()), 300])
  for i in range(len(X_ws_train_list[j].split())):
    try:
      X_ws_train_ft_vectors[i] = fasttext_vectors.wv[X_ws_train_list[j].split()[i]]
    except KeyError:
      X_ws_train_ft_vectors[i] = np.zeros(300)
  X_ws_train_ft_max_vectors[j] = X_ws_train_ft_vectors.max(axis=0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [ ]:
X_ws_train_ft_minmax_vectors = np.concatenate((X_ws_train_ft_min_vectors, X_ws_train_ft_max_vectors), axis=1)

In [ ]:
X_ws_train_ft_minmax_vectors.shape

(13705, 600)

In [ ]:
scaler = StandardScaler()
X_ws_train_ft_minmax_vectors = scaler.fit_transform(X_ws_train_ft_minmax_vectors)

Doing the same for the validation set.

In [ ]:
X_ws_valid_ft_min_vectors = np.empty([len(X_ws_valid_list), 300])
for j in range(len(X_ws_valid_list)):
  X_ws_valid_ft_vectors = np.empty([len(X_ws_valid_list[j].split()), 300])
  for i in range(len(X_ws_valid_list[j].split())):
    try:
      X_ws_valid_ft_vectors[i] = fasttext_vectors.wv[X_ws_valid_list[j].split()[i]]
    except KeyError:
      X_ws_valid_ft_vectors[i] = np.zeros(300)
  X_ws_valid_ft_min_vectors[j] = X_ws_valid_ft_vectors.min(axis=0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [ ]:
X_ws_valid_ft_max_vectors = np.empty([len(X_ws_valid_list), 300])
for j in range(len(X_ws_valid_list)):
  X_ws_valid_ft_vectors = np.empty([len(X_ws_valid_list[j].split()), 300])
  for i in range(len(X_ws_valid_list[j].split())):
    try:
      X_ws_valid_ft_vectors[i] = fasttext_vectors.wv[X_ws_valid_list[j].split()[i]]
    except KeyError:
      X_ws_valid_ft_vectors[i] = np.zeros(300)
  X_ws_valid_ft_max_vectors[j] = X_ws_valid_ft_vectors.min(axis=0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [ ]:
X_ws_valid_ft_minmax_vectors = np.concatenate((X_ws_valid_ft_min_vectors, X_ws_valid_ft_max_vectors), axis=1)

In [ ]:
scaler = StandardScaler()
X_ws_valid_ft_minmax_vectors = scaler.fit_transform(X_ws_valid_ft_minmax_vectors)

In [ ]:
dense_model_mm = tf.keras.models.Sequential()
dense_model_mm.add(tf.keras.Input(shape=(600,)))
dense_model_mm.add(tf.keras.layers.Dense(300, activation='relu', kernel_initializer='he_normal'))
dense_model_mm.add(tf.keras.layers.Dense(300, activation='relu', kernel_initializer='he_normal'))
dense_model_mm.add(tf.keras.layers.Dense(300, activation='relu', kernel_initializer='he_normal'))
dense_model_mm.add(tf.keras.layers.Dense(3, activation='softmax'))
dense_model_mm.output_shape

(None, 3)

In [ ]:
dense_model_mm.compile(optimizer=Adam(lr=1e-5),loss='categorical_crossentropy' ,metrics=['accuracy'])

In [ ]:
dense_model_mm.fit(X_ws_train_ft_minmax_vectors, y_ws_train_list,  validation_split = 0.2,
                     callbacks=[early_stopping], epochs=75)

Epoch 1/75
343/343 [==============================] - 2s 5ms/step - loss: 1.2236 - accuracy: 0.3841 - val_loss: 1.1237 - val_accuracy: 0.4279
Epoch 2/75
343/343 [==============================] - 2s 5ms/step - loss: 1.0938 - accuracy: 0.4479 - val_loss: 1.0537 - val_accuracy: 0.4798
Epoch 3/75
343/343 [==============================] - 2s 5ms/step - loss: 1.0214 - accuracy: 0.4930 - val_loss: 1.0106 - val_accuracy: 0.5086
Epoch 4/75
343/343 [==============================] - 2s 5ms/step - loss: 0.9697 - accuracy: 0.5293 - val_loss: 0.9812 - val_accuracy: 0.5265
Epoch 5/75
343/343 [==============================] - 2s 5ms/step - loss: 0.9290 - accuracy: 0.5586 - val_loss: 0.9563 - val_accuracy: 0.5469
Epoch 6/75
343/343 [==============================] - 2s 6ms/step - loss: 0.8944 - accuracy: 0.5838 - val_loss: 0.9384 - val_accuracy: 0.5556
Epoch 7/75
343/343 [==============================] - 2s 5ms/step - loss: 0.8649 - accuracy: 0.6044 - val_loss: 0.9248 - val_accuracy: 0.5669
Epoch 

In [ ]:
score = dense_model_mm.evaluate(X_ws_valid_ft_minmax_vectors, y_ws_valid_list, verbose = 0)
print("Test score: ", score[0])
print("Test accuracy: ", score[1])

Test score:  1.4225778579711914
Test accuracy:  0.4877426028251648


### CNN model by Dani

Only using sentences containing 100 or fewer words:

In [ ]:
cutoff = 100

In [ ]:
def shorten(X_input_sentences, y_input_labels, length):
    X_out = []
    y_out = []
    for i in range(len(X_input_sentences)):
        if len(X_input_sentences[i].split(' ')) <= length:
            X_out.append(X_input_sentences[i])
            y_out.append(y_input_labels[i])
    return X_out, np.array(y_out)

X_ws_train_list_short, y_ws_train_list_short = shorten(X_ws_train_list,y_ws_train_list,cutoff)
X_ws_valid_list_short, y_ws_valid_list_short = shorten(X_ws_valid_list,y_ws_valid_list,cutoff)

Filling out 100-long senteces by repeating the words in the sentence, e.g. for sentence "We have talked." we get: [we have talked we have talked we .... have talked we]

In [ ]:
def set_of_sentences_to_vec(orig_list, vec_length, cutoff):

    out = np.empty([len(orig_list), cutoff, vec_length])

    for j in range(len(orig_list)):
        add_vectors = np.empty([cutoff, vec_length])
        
        sentence_length = len(orig_list[j].split())
        
        for i in range(cutoff):
            try:
                add_vectors[i] = fasttext_vectors.wv[orig_list[j].split()[i % sentence_length]]
            except KeyError:
                add_vectors[i] = np.zeros(vec_length)

        out[j] = add_vectors
  
    return out

X_ws_train_ft_all_vectors = set_of_sentences_to_vec(X_ws_train_list_short, 300, cutoff)
X_ws_valid_ft_all_vectors = set_of_sentences_to_vec(X_ws_valid_list_short, 300, cutoff)

C:\Users\AMD\anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':


In [ ]:
X_ws_train_ft_all_vectors.shape

(13640, 100, 300)

In [ ]:
X_ws_valid_ft_all_vectors.shape

(5850, 100, 300)

In [ ]:
X_ws_train_ft_all_vectors_tp = np.transpose(X_ws_train_ft_all_vectors,axes=(0,2,1))
X_ws_valid_ft_all_vectors_tp = np.transpose(X_ws_valid_ft_all_vectors,axes=(0,2,1))

In [ ]:
X_ws_train_ft_all_vectors_tp.shape

(13640, 300, 100)

In [ ]:
X_ws_valid_ft_all_vectors_tp.shape

(5850, 300, 100)

Simple convolutional model with kernel size 7 that goes through the sentence:

In [ ]:
conv_model = tf.keras.models.Sequential()
conv_model.add(tf.keras.layers.Conv1D(300,kernel_size=7,
        input_shape=(100,300), activation='relu',
        kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3),
        bias_regularizer=regularizers.l2(1e-3),
        activity_regularizer=regularizers.l2(1e-4)))
conv_model.add(tf.keras.layers.Flatten())
conv_model.add(tf.keras.layers.Dense(94, activation='relu',
        kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3),
        bias_regularizer=regularizers.l2(1e-3),
        activity_regularizer=regularizers.l2(1e-4)))
conv_model.add(tf.keras.layers.Dropout(0.4))
conv_model.add(tf.keras.layers.Dense(3, activation='softmax'))
conv_model.output_shape

(None, 3)

In [ ]:
conv_model.summary()

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_169 (Conv1D)          (None, 94, 300)           630300    
_________________________________________________________________
flatten_51 (Flatten)         (None, 28200)             0         
_________________________________________________________________
dense_121 (Dense)            (None, 64)                1804864   
_________________________________________________________________
dropout_66 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_122 (Dense)            (None, 3)                 195       
Total params: 2,435,359
Trainable params: 2,435,359
Non-trainable params: 0
_________________________________________________________________


In [ ]:
conv_model.compile(optimizer='adam',loss='categorical_crossentropy' ,metrics=['accuracy'])

More complicated convolutional model that uses 4 different kernel sizes, then concatenates them and finishes with a single Dropout and Dense layer:

In [ ]:
inp = tf.keras.Input(shape=(300,100), name='input')

def conv_pool_layer(kernel_size,pool_size,input_layer,name):
    
    conv = tf.keras.layers.Conv1D(100,kernel_size=kernel_size,
        input_shape=(300,100), activation='relu', padding='same',
        kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3),
        bias_regularizer=regularizers.l2(1e-3),
        activity_regularizer=regularizers.l2(1e-4),
        data_format='channels_last', name='1D_conv' + name)(input_layer)
    
    pool = max_pool3 = tf.keras.layers.MaxPooling1D(pool_size=pool_size,
        data_format='channels_first', name= '1D_maxpool' + name)(conv)

    return pool

model3 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(3,2,inp,'3'))
model5 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(5,2,inp,'5'))
model7 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(7,4,inp,'7'))
model9 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(9,4,inp,'9'))

combined = tf.keras.layers.concatenate([model3.output, model5.output,
                                        model7.output, model9.output], name='concatenate')

flat = tf.keras.layers.Flatten(name='flatten')(combined)
drop = tf.keras.layers.Dropout(0.57,name='droput_0.5')(flat)
dense = tf.keras.layers.Dense(3, activation='softmax',name='dense')(drop)

conv_model2 = tf.keras.Model(inputs=[model3.input],
                            outputs=dense,name='Multiconv_Model')
conv_model2.output_shape

(None, 3)

In [ ]:
conv_model2.summary()

Model: "Multiconv_Model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 300, 100)]   0                                            
__________________________________________________________________________________________________
1D_conv3 (Conv1D)               (None, 300, 100)     30100       input[0][0]                      
__________________________________________________________________________________________________
1D_conv5 (Conv1D)               (None, 300, 100)     50100       input[0][0]                      
__________________________________________________________________________________________________
1D_conv7 (Conv1D)               (None, 300, 100)     70100       input[0][0]                      
____________________________________________________________________________________

In [ ]:
conv_model2.compile(optimizer='adam',loss='categorical_crossentropy' ,metrics=['accuracy'])

In [ ]:
for i in range(8):
    print("SESSION " + str(i)) 
    conv_model.fit(X_ws_train_ft_all_vectors, y_ws_train_list_short,
                   epochs=5, batch_size=20, validation_split=1/11)
    conv_model.save('author_identification_conv2_model' + str(i) + '.h5')

SESSION 0
Train on 12400 samples, validate on 1240 samples
Epoch 1/5
  320/12400 [..............................] - ETA: 9:02 - loss: 1.0007 - acc: 0.5656

KeyboardInterrupt: 

Teaching this Neural Net took ~8 hours for 30 epochs, so it has been interrupted multiple times and saved into a separate Keras H5 file.

In [ ]:
for i in range(8):
    print("SESSION " + str(i)) 
    conv_model2.fit(X_ws_train_ft_all_vectors_tp, y_ws_train_list_short,
                   epochs=5, batch_size=20, validation_split=1/11)
    conv_model2.save('author_identification_multiconv2_model' + str(i) + '.h5')

SESSION 0
Train on 12400 samples, validate on 1240 samples
Epoch 1/5
12400/12400 [==============================] - 961s 77ms/sample - loss: 1.3436 - acc: 0.5388 - val_loss: 0.8882 - val_acc: 0.6621
Epoch 2/5
12400/12400 [==============================] - 940s 76ms/sample - loss: 0.8725 - acc: 0.6653 - val_loss: 0.8531 - val_acc: 0.6589
Epoch 3/5
12400/12400 [==============================] - 953s 77ms/sample - loss: 0.8173 - acc: 0.6938 - val_loss: 0.8310 - val_acc: 0.6710
Epoch 4/5
12400/12400 [==============================] - 929s 75ms/sample - loss: 0.7828 - acc: 0.7198 - val_loss: 0.8635 - val_acc: 0.6476
Epoch 5/5
12400/12400 [==============================] - 940s 76ms/sample - loss: 0.7582 - acc: 0.7296 - val_loss: 0.8605 - val_acc: 0.6815
SESSION 1
Train on 12400 samples, validate on 1240 samples
Epoch 1/5
12400/12400 [==============================] - 789s 64ms/sample - loss: 0.7487 - acc: 0.7435 - val_loss: 0.8902 - val_acc: 0.6935
Epoch 2/5
12400/12400 [===================

KeyboardInterrupt: 

In [ ]:
conv_model2 = tf.keras.models.load_model('author_identification_multiconv2_model1.h5')

In [ ]:
for i in range(2,8):
    print("SESSION " + str(i)) 
    conv_model2.fit(X_ws_train_ft_all_vectors_tp, y_ws_train_list_short,
                   epochs=5, batch_size=20, validation_split=1/11)
    conv_model2.save('author_identification_multiconv2_model' + str(i) + '.h5')

SESSION 2
Train on 12400 samples, validate on 1240 samples
Epoch 1/5
12400/12400 [==============================] - 852s 69ms/sample - loss: 0.6209 - acc: 0.8281 - val_loss: 1.0151 - val_acc: 0.6806
Epoch 2/5
12400/12400 [==============================] - 964s 78ms/sample - loss: 0.6006 - acc: 0.8428 - val_loss: 0.9591 - val_acc: 0.6823
Epoch 3/5
12400/12400 [==============================] - 1042s 84ms/sample - loss: 0.5900 - acc: 0.8441 - val_loss: 0.9958 - val_acc: 0.6677
Epoch 4/5
12400/12400 [==============================] - 842s 68ms/sample - loss: 0.5723 - acc: 0.8552 - val_loss: 1.0274 - val_acc: 0.6766
Epoch 5/5
12400/12400 [==============================] - 823s 66ms/sample - loss: 0.5647 - acc: 0.8643 - val_loss: 1.0167 - val_acc: 0.6815
SESSION 3
Train on 12400 samples, validate on 1240 samples
Epoch 1/5
12400/12400 [==============================] - 812s 65ms/sample - loss: 0.5666 - acc: 0.8723 - val_loss: 1.1128 - val_acc: 0.6629
Epoch 2/5
12400/12400 [==================

KeyboardInterrupt: 

In [ ]:
conv_model2 = tf.keras.models.load_model('author_identification_multiconv2_model5.h5')

In [ ]:
with open('X_ws_train_ft_all_vectors.npy', 'wb') as f:
    np.save(f, X_ws_train_ft_all_vectors)
with open('X_ws_train_ft_all_vectors_tp.npy', 'wb') as f:
    np.save(f, X_ws_train_ft_all_vectors_tp)
with open('y_ws_train_list_short.npy', 'wb') as f:
    np.save(f, y_ws_train_list_short)
with open('X_ws_valid_ft_all_vectors.npy', 'wb') as f:
    np.save(f, X_ws_valid_ft_all_vectors)
with open('X_ws_valid_ft_all_vectors_tp.npy', 'wb') as f:
    np.save(f, X_ws_valid_ft_all_vectors_tp)
with open('y_ws_valid_list_short.npy', 'wb') as f:
    np.save(f, y_ws_valid_list_short)

In [ ]:
'''
with open('X_ws_train_ft_all_vectors.npy', 'rb') as f:
    X_ws_train_ft_all_vectors = np.load(f)
'''

In [ ]:
conv_model.evaluate(X_ws_valid_ft_all_vectors, y_ws_valid_list_short)

5850/5850 [==============================] - 172s 29ms/sample - loss: 1.0111 - acc: 0.5489


[1.0110511044151762, 0.54888886]

In [ ]:
conv_model2.evaluate(X_ws_valid_ft_all_vectors_tp, y_ws_valid_list_short)

5850/5850 [==============================] - 197s 34ms/sample - loss: 1.2288 - acc: 0.6598


[1.2288284897192931, 0.6598291]

In [ ]:
saved_model = tf.keras.models.load_model('author_identification_conv_model5.h5')

In [ ]:
saved_model.evaluate(X_ws_valid_ft_all_vectors, y_ws_valid_list_short)

5850/5850 [==============================] - 131s 22ms/sample - loss: 1.5943 - acc: 0.6821


[1.5943330551620223, 0.6820513]

In [ ]:
saved_model2 = tf.keras.models.load_model('author_identification_multiconv2_model1.h5')

Unfortunately the above model didn't give as good of an accuracy as the simple Dense model, even through we've tried many parameter settings:
- Chaning the number of Convolutional Layers and how they connect
- Adding or removing Dense Layers at the end
- Changing the Dropout rate
- Adding regularization
- Changing the structure of the Neural Net.

We'll do some hyperparameter-optimization to get a better accuracy later.

In [ ]:
saved_model2.evaluate(X_ws_valid_ft_all_vectors_tp, y_ws_valid_list_short)

5850/5850 [==============================] - 181s 31ms/sample - loss: 0.9471 - acc: 0.6858


[0.9470657617821653, 0.68581194]